# 단어 변환

두 단어 begin, target 단어 집함 words
begin => target 변환하는 가장 짧은 변환 과정 찾기.

1. 한 번에 한 개의 알파벳만 바꿀 수 있다.
2. words에 있는 단어로만 변환할 수 있다.

예를 들어 begin이 hit, target가 cog, words가 [hot,dot,dog,lot,log,cog]라면 hit -> hot -> dot -> dog -> cog와 같이 4단계를 거쳐 변환할 수 있습니다.

두 개의 단어 begin, target과 단어의 집합 words가 매개변수로 주어질 때, 최소 몇 단계의 과정을 거쳐 begin을 target으로 변환할 수 있는지 return 하도록 solution 함수 작성

- 각 단어는 소문자
- 각 단어 길이 3이상 10이하, 모든 단어 길이 같음
- words에는 3개 이상 50개 이하 단어
- begin, target은 같지 않다.
- 변환 불가하면 0 return

# 시도 1: 2,3,4 번 시간초과
1. target이 words 안에 없다면 return 0
2. words 로 주어진 목록을 토대로 graph를 만든다.
3. 첫 level에 begin 문자열을 넣고 시작
4. 각 level 에서 current 값을 하나씩 꺼낸다.
5. current에서 한 글자 변경해서 만들 수 있는 문자열들을 다음 레벨에 포함시키고 words 목록에서 제거.
6. 그리고 만들어진 다음레벨에서 순서대로 4번부터 다시 진행하는 구조.
7. words 에 있는 문자열들을 다 사용하면 끝낸다.
8. 그리고 만들어진 graph에서 target을 대상으로 dfs 진행하고, visited 에 포함된 갯수 -1 이 답 ( begin 제외 )

In [26]:
from collections import defaultdict
def solution(begin, target, words):
    answer = 0
    
    if target not in words:
        return answer
    
    graph = defaultdict(str)

    def one_char_diff(s1, s2):
        count = 0

        for c1, c2 in zip(s1, s2):
            if c1 != c2:
                if count == 1:
                    return False
                else:
                    count+=1

        return True
     
    level = [begin]

    while len(words):    
        next_level = set()

        for current in level:
            # 중복되는 단어가 없기 때문에 set을 사용해도 된다.
            graph[current] = set()        

            for w in words:
                if one_char_diff(current,w):
                    graph[current].add(w)
                    next_level.add(w)            
            
            for sub in graph[current]:
                words.remove(sub)

        level = next_level

    # print(graph)
    
    def DFS_count_with_adj_list(graph, root):
        count = 0
        visited = []
        stack = [root]

        while stack:
            n = stack.pop()
            if n not in visited:
                visited.append(n)                
                # print(n,graph[n],target)
                if n == target:
                    break
                stack += graph[n] - set(visited)    
                count+=1                                                            

        return count

    answer = DFS_count_with_adj_list(graph,begin)
    
    return answer

In [27]:
solution('hit','cog',['hot', 'dot', 'dog', 'lot', 'log', 'cog'])

4

# 시도 2:
최소 경로 찾기 접근.   
1. words에 들어있는 단어들 + begin 단어를 대상으로, 한 글자 차이가 나는 경우 연결되는 graph 생성   
2. 1에서 생성된 그래프에서 시작점을 begin 단어로 놓고, target 단어로 가는 최소 경로를 탐색.

In [35]:
import heapq
from collections import defaultdict
def solution(begin, target, words):
    answer = 0
    
    # target이 words 안에 없으면 끝
    if target not in words:
        return answer
    
    graph = defaultdict(str)

    whole_words = words.copy()
    whole_words.insert(0,begin)

    # print(whole_words)

    def one_char_diff(s1, s2):
        count = 0

        for c1, c2 in zip(s1, s2):
            if c1 != c2:
                if count == 1:
                    return False
                else:
                    count+=1

        return True

    # N = len(words) 3이상 50이하
    # O(N^2) => 2500 max
    for idx,word in enumerate(whole_words):
        graph[word] = defaultdict(str)    
        for inner_idx,inner_word in enumerate(whole_words):

            if inner_idx == idx:
                continue
            else:
                if one_char_diff(word,inner_word):
                    graph[word][inner_word]=1

    # print(graph)


    # 탐색할 그래프와 시작 정점을 인수로 전달받습니다.
    def dijkstra(graph, start, end):
        # 시작 정점에서 각 정점까지의 거리를 저장할 딕셔너리를 생성하고, 무한대(inf)로 초기화합니다.
        distances = {vertex: [float('inf'), start] for vertex in graph}

        # 그래프의 시작 정점의 거리는 0으로 초기화 해줌
        distances[start] = [0, start]

        # 모든 정점이 저장될 큐를 생성합니다.
        queue = []

        # 그래프의 시작 정점과 시작 정점의 거리(0)을 최소힙에 넣어줌
        heapq.heappush(queue, [distances[start][0], start])

        while queue:
            # 가장 가까운 거리에 있는 정점들을 꺼내는거지 ㅇㅇ
            # 큐에서 정점을 하나씩 꺼내 인접한 정점들의 가중치를 모두 확인하여 업데이트합니다.
            current_distance, current_vertex = heapq.heappop(queue)

    # 더 짧은 경로가 있다면 무시한다.
                    # 현재 대상이 되고 있는 정점에 대해,
    #시작 정점으로부터의 거리(distances)가 heap에서 추출된 거리보다 짧다면 걍 넘어간다는거지.
                
            if distances[current_vertex][0] < current_distance:
                continue
                
            for adjacent, weight in graph[current_vertex].items():
                distance = current_distance + weight
                # 만약 시작 정점에서 인접 정점으로 바로 가는 것보다 (지금까지 기록된 최단거리인거지)
                            # 현재 정점을 통해 가는 것이 더 가까울 경우에는 갱신을 해야한다. (거리 업데이트)
                if distance < distances[adjacent][0]:
                    # 거리를 업데이트합니다.
                    distances[adjacent] = [distance, current_vertex]
                    heapq.heappush(queue, [distance, adjacent])
        
            path = end
        path_output = end + '->'
        count = 1
        while distances[path][1] != start:
            count +=1
            path_output += distances[path][1] + '->'
            path = distances[path][1]
        path_output += start
        # print (path_output)

        return count,distances

    # # 방향 그래프
    # mygraph = {
    #     'A': {'B': 8, 'C': 1, 'D': 2},
    #     'B': {},
    #     'C': {'B': 5, 'D': 2},
    #     'D': {'E': 3, 'F': 5},
    #     'E': {'F': 1},
    #     'F': {'A': 5}
    # }

    # print(dijkstra(mygraph, 'A', 'F'))
    # print(dijkstra(graph, 'hit', 'cog'))
    c,d  = dijkstra(graph, 'hit', 'cog')
    # print('r',c,d)
    answer = c

    
    return answer

In [36]:
solution('hit','cog',['hot', 'dot', 'dog', 'lot', 'log', 'cog'])

4

In [16]:
import heapq
# 탐색할 그래프와 시작 정점을 인수로 전달받습니다.
def dijkstra(graph, start, end):
    # 시작 정점에서 각 정점까지의 거리를 저장할 딕셔너리를 생성하고, 무한대(inf)로 초기화합니다.
    distances = {vertex: [float('inf'), start] for vertex in graph}

    # 그래프의 시작 정점의 거리는 0으로 초기화 해줌
    distances[start] = [0, start]

    # 모든 정점이 저장될 큐를 생성합니다.
    queue = []

    # 그래프의 시작 정점과 시작 정점의 거리(0)을 최소힙에 넣어줌
    heapq.heappush(queue, [distances[start][0], start])

    while queue:
        # 가장 가까운 거리에 있는 정점들을 꺼내는거지 ㅇㅇ
        # 큐에서 정점을 하나씩 꺼내 인접한 정점들의 가중치를 모두 확인하여 업데이트합니다.
        current_distance, current_vertex = heapq.heappop(queue)

# 더 짧은 경로가 있다면 무시한다.
                # 현재 대상이 되고 있는 정점에 대해,
#시작 정점으로부터의 거리(distances)가 heap에서 추출된 거리보다 짧다면 걍 넘어간다는거지.
            
        if distances[current_vertex][0] < current_distance:
            continue
            
        for adjacent, weight in graph[current_vertex].items():
            distance = current_distance + weight
            # 만약 시작 정점에서 인접 정점으로 바로 가는 것보다 (지금까지 기록된 최단거리인거지)
                        # 현재 정점을 통해 가는 것이 더 가까울 경우에는 갱신을 해야한다. (거리 업데이트)
            if distance < distances[adjacent][0]:
                # 거리를 업데이트합니다.
                distances[adjacent] = [distance, current_vertex]
                heapq.heappush(queue, [distance, adjacent])
    
        path = end
    path_output = end + '->'
    while distances[path][1] != start:
        path_output += distances[path][1] + '->'
        path = distances[path][1]
    path_output += start
    print (path_output)

    return distances
# 방향 그래프
mygraph = {
    'A': {'B': 8, 'C': 1, 'D': 2},
    'B': {},
    'C': {'B': 5, 'D': 2},
    'D': {'E': 3, 'F': 5},
    'E': {'F': 1},
    'F': {'A': 5}
}

print(dijkstra(mygraph, 'A', 'F'))

F->E->D->A
{'A': [0, 'A'], 'B': [6, 'C'], 'C': [1, 'A'], 'D': [2, 'A'], 'E': [5, 'D'], 'F': [6, 'E']}


# 시도 3 :

In [1]:
def one_char_diff(s1, s2):
        count = 0

        for c1, c2 in zip(s1, s2):
            if c1 != c2:
                if count == 1:
                    return False
                else:
                    count+=1

        return True

def bfs(begin, target, words, visited, answer):
    queue = []
    queue.append(begin)
    while queue:
        word = queue.pop()

        if word == target:
            return answer

        for i in range(0, len(words)):
            # 글자수가 하나만 다른 단어 찾기
            if one_char_diff(words[i], word):
                if visited[i] == 1:
                    continue
                visited[i] = 1
                # next level 같은 개념으로 생각하지 않고, 그냥 queue 에 추가하고, queue에서 다시 뽑아내는 방식으로 하는게 훨씬 간단하고 효율적이다!
                queue.append(words[i])
        answer += 1


def solution(begin, target, words):
    answer = 0
    if target not in words:
        return 0

    visited = [0] * len(words)

    answer = bfs(begin, target, words, visited, answer)

    return answer

In [2]:
solution('hit','cog',['hot', 'dot', 'dog', 'lot', 'log', 'cog'])

4